In [1]:
import pandas as pd
import numpy as np
import boto3
import requests
import json
import configparser
import psycopg2

import awswrangler as wr
import redshift_connector

In [2]:
import sql_statements

### Get Credentials and S3 Client

In [3]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

['dwh.cfg']

In [4]:
s3 = boto3.resource('s3', 
                    aws_access_key_id = config['S3']['AWS_ACCESS_KEY_ID'], 
                    aws_secret_access_key = config['S3']['AWS_SECRET_ACCESS_KEY'])
s3_client = boto3.client('s3', 
                    aws_access_key_id = config['S3']['AWS_ACCESS_KEY_ID'], 
                    aws_secret_access_key = config['S3']['AWS_SECRET_ACCESS_KEY'])

### Download Contest JSON and Upload to S3

### Get Payout Data and Upload to S3

### Connect to Redshift

In [5]:
#conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
conn = redshift_connector.connect(
    host=config['CLUSTER']['HOST'],
    database=config['CLUSTER']['DB_NAME'],
    user=config['CLUSTER']['DB_USER'],
    password=config['CLUSTER']['DB_PASSWORD'],
 )

In [6]:
cur = conn.cursor()

###  Create/Clean and Upload Contest Entry

### Contest

In [ ]:
#CREATE Table
cur.execute(sql_statements.contest_table_create)
conn.commit()

#Upload to Table
cur.execute(sql_statements.contest_table_copy)
conn.commit()

### Payout Table 

In [ ]:
cur.execute(sql_statements.payout_table_create)
conn.commit()

In [ ]:
cur.execute(sql_statements.payout_table_copy)
conn.commit()

### Player Lineups

In [ ]:
cur.execute(sql_statements.player_lineups_create)
conn.commit()

In [ ]:
cur.execute(sql_statements.player_lineup_insert)
conn.commit()

In [18]:
#Data Quality Check

player_linup_contests = cur.execute("""
    SELECT count(distinct contestid)
    FROM player_lineup
""").fetchall()[0][0]

print("There are {} contests in player_lineup table".format(player_linup_contests))

There are 1771 contests in player_lineup table


### Player Payout

In [7]:
cur.execute(sql_statements.player_payout_create)
conn.commit()

In [8]:
cur.execute(sql_statements.player_payout_insert)
conn.commit()

In [17]:
#Data Quality Check

player_payout_contests = cur.execute("""
    SELECT count(distinct contestid)
    FROM player_payout
""").fetchall()[0][0]

print("There are {} contests in player_payout table".format(player_payout_contests))

There are 1771 contests in player_payout table
